
### Logs
* 30.08: use 2048 for max_len and max_position_embeddings
* 03.09: use 512 instead
* 23.09: 512 pretrained with both datasets in MLM. training with 2048 and try to infer with 512. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* 25.09 512 pretrained with both datasets in MLM. training with 1536. Pretraining notebook: pretrain1_task_ver3(512_larger dataset)
* no pretraining
* no pretraining use max_length as 1600, learning rate change to 2e-5
* no pretraining use max_length as 1600, learning rate changed to 4e-6, cosine scheduler
* no pretraining use max_legnth as 1600, learning rate changed back to 2e-6, cosine scheduler
* change weight decay from 0.01 to 1e-4, learning rate changed to 4e-6
* pretraining 1009, weight decay 1e-4, learning reate 4e-6
* pretraining 2109, weight decay 1e-4, learning reate 4e-6
* batch_size 8, eval batch_size 16

In [2]:
!pip install transformers -q
!pip install sentencepiece -q
!pip install datasets -q
!pip install accelerate -U -q

In [3]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

from datasets import Dataset, disable_progress_bar
import pandas as pd

pdf = pd.read_csv("./input/prompts_train.csv")
sdf = pd.read_csv("./input/summaries_train.csv")

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /microsoft/deberta-v3-large/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fa9e697ffa0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 51c07663-125a-405d-a826-b0e4cf7a2da5)')' thrown while requesting HEAD https://huggingface.co/microsoft/deberta-v3-large/resolve/main/tokenizer_config.json
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/root/miniconda3/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have

In [4]:
from transformers import AutoModelForSequenceClassification
#model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large")
## use the pretrained model
# from transformers import AutoConfig
# config = AutoConfig.from_pretrained('./input/pretrain/pretrained_model/')
# model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model', config = config)

## Train

In [14]:
%%writefile train.py

import os
import logging
import warnings
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    set_seed,
    Trainer,
    TrainingArguments,
    HfArgumentParser,
    DataCollatorWithPadding,
)
from datasets import Dataset, disable_progress_bar
import pandas as pd
import numpy as np

warnings.simplefilter("ignore")
logging.disable(logging.ERROR)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['WANDB_PROJECT'] = 'kaggle-commonlit-eval-student-summaries-2609-no-pretrain'

disable_progress_bar()

@dataclass
class Config:
    model_name_or_path: Optional[str] = field(
        default="microsoft/deberta-v3-large",
        metadata={"help": "Model name or path"},
    )

    data_dir: Optional[str] = field(
        default="/kaggle/input/commonlit-evaluate-student-summaries",
        metadata={"help": "Data directory"},
    )

    max_seq_length: Optional[int] = field(
        default=1600,
        #default = 512,
        #default = 512,
        metadata={"help": "Max sequence length"},
    )

    add_prompt_question: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt question into input"},
    )

    add_prompt_text: Optional[bool] = field(
        default=False,
        metadata={"help": "Add prompt text into input"},
    )

    fold: Optional[int] = field(
        default=0,
        metadata={"help": "Fold"},
    )

    num_proc: Optional[int] = field(
        default=4,
        metadata={"help": "Number of processes"},
    )

    dropout: Optional[float] = field(
        default=0.,
        metadata={"help": "Amount of dropout to apply"},
    )
    max_position_embeddings: Optional[int] = field(
        default=1600,
        #default=512,
        #default = 512,
        metadata={"help": "Amount of dropout to apply"},
    )


# Spell auto correction
# from spellchecker import SpellChecker

# def correct_spelling(input_text):
#     print('input_text: ', input_text)
#     # Initialize the spell checker
#     spell = SpellChecker()
#     # Split the input text into words
#     words = input_text.split()
#     # Initialize an empty list to store the corrected words
#     corrected_words = []
#     for word in words:
#         # Check if the word has any punctuation at the end
#         if word[-1].isalpha():
#             # Extract the punctuation
#             punctuation_end = ""
#         else:
#             punctuation_end = word[-1]
#             word = word[:-1]
#         # check if the word has any punctuation at the start
#         if word[0].isalpha():
#            # Extract the punctuation
#             punctuation_start = ""
#         else:
#             punctuation_start = word[0]
#             word = word[1:]
#         # Check the spelling of the word (case insensitive)
#         corrected_word = spell.correction(word.lower())
#         # Preserve the original capitalization
#         if word[0].isupper():
#             corrected_word = corrected_word.capitalize()
#         # Combine the corrected word and punctuation (if any)
#         corrected_word = punctuation_start+corrected_word+punctuation_end
#         # Append the corrected word to the list
#         corrected_words.append(corrected_word)
#     # Join the corrected words back into a single string
#     corrected_text = " ".join(corrected_words)
#     return corrected_text

def tokenize(example, tokenizer, config):
    sep = tokenizer.sep_token

    # if config.add_prompt_question:
    #     text = sep.join(
    #         [example["prompt_question"], example["prompt_text"], example["text"]]
    #     )
    # elif config.add_prompt_text:
    #     text = sep.join([example["prompt_text"], example["text"]])
    # else:
    #     text = example["text"]
    prompt = sep.join([example["prompt_title"], example["prompt_text"], example["prompt_question"]])
    labels = [example["content"], example["wording"]]

    tokenized = tokenizer(
#         prompt,
#         example["text"],
        example['text'],
        prompt,
        padding=False,
        truncation=True, # changed by Peng, turn on the truncation
        max_length=config.max_seq_length,
    )

    return {
        **tokenized,
        "labels": labels,
    }




def compute_mcrmse(eval_pred):
    """
    Calculates mean columnwise root mean squared error
    https://www.kaggle.com/competitions/commonlit-evaluate-student-summaries/overview/evaluation
    """
    preds, labels = eval_pred

    col_rmse = np.sqrt(np.mean((preds - labels) ** 2, axis=0))
    mcrmse = np.mean(col_rmse)

    return {
        "content_rmse": col_rmse[0],
        "wording_rmse": col_rmse[1],
        "mcrmse": mcrmse,
    }


def main():
    parser = HfArgumentParser((Config, TrainingArguments))

    config, training_args = parser.parse_args_into_dataclasses()

    set_seed(training_args.seed)

    if "wandb" in training_args.report_to:
        import wandb

        try:
#             from kaggle_secrets import UserSecretsClient
#             user_secrets = UserSecretsClient()
#             key = user_secrets.get_secret("wandb")

#             wandb.login(key=key)
            wandb.login()
        except:
            print("Could not log in to WandB")

    tokenizer = AutoTokenizer.from_pretrained(config.model_name_or_path)
    model_config = AutoConfig.from_pretrained(config.model_name_or_path)

    model_config.update({
        "hidden_dropout_prob": config.dropout,
        "attention_probs_dropout_prob": config.dropout,
        "num_labels": 2,
        "problem_type": "regression",
        "max_position_embeddings": config.max_position_embeddings,
        "cfg": config.__dict__,
    })

    print(model_config)
    
    # Do not use pretrained model
    model = AutoModelForSequenceClassification.from_pretrained(
        config.model_name_or_path, config=model_config
    )
    
    # use pretrained model
    #print('use pretrained_model')
    #print('./input/pretrain/pretrained_model_2109/')
    #model = AutoModelForSequenceClassification.from_pretrained('./input/pretrain/pretrained_model_2109/', config = model_config)

    #pdf = pd.read_csv(f"{config.data_dir}/prompts_train.csv")
    pdf = pd.read_csv(f"./input/prompts_train.csv")
    #sdf = pd.read_csv(f"{config.data_dir}/summaries_train.csv")
    sdf = pd.read_csv(f"./input/summaries_train.csv")

    df = pdf.merge(sdf, on="prompt_id")

    # 4 prompt ids, 4 folds
    id2fold = {
        "814d6b": 0,
        "39c16e": 1,
        "3b9047": 2,
        "ebad26": 3,
    }

    df["fold"] = df["prompt_id"].map(id2fold)

    train_ds = Dataset.from_pandas(df[df["fold"] != config.fold])
    val_ds = Dataset.from_pandas(df[df["fold"] == config.fold])

    train_ds = train_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    val_ds = val_ds.map(
        tokenize,
        batched=False,
        num_proc=config.num_proc,
        fn_kwargs={"tokenizer": tokenizer, "config": config},
    )

    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        pad_to_multiple_of=16 if training_args.fp16 else None,
    )

    training_args.bf16 =True
    training_args.gradient_accumulation_steps = 1
    training_args.load_best_model_at_end = True
    training_args.greater_is_better = False
    training_args.metric_for_best_model = 'mcrmse'
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_mcrmse,
    )

    trainer.train()

    model.config.best_metric = trainer.state.best_metric
    model.config.save_pretrained(training_args.output_dir)

    trainer.log({"eval_best_mcrmse": trainer.state.best_metric})


if __name__ == "__main__":
    main()

Overwriting train.py


In [15]:
from pathlib import Path

seed = 42

fold = 1

output = f"output_fold{fold}_seed{seed}_2609"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 4e-6 \
  --weight_decay 1e-4 \
  --warmup_ratio 0 \
  --lr_scheduler_type "cosine" \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 1,
    "max_position_embeddings": 1600,
    "max_seq_length": 1600,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1600,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'eval_loss': 0.2840399742126465, 'eval_content_rmse': 0.5003312230110168, 'eval_wording_rmse': 0.5636917948722839, 'eval_mcrmse': 0.5320115089416504, 'eval_runtime': 112.1194, 'eval_samples_per_second': 18.347, 'eval_steps_per_second': 9.178, 'epoch': 0.29}
{'loss': 0.2414, 'learning_rate': 3.94619475478414e-06, 'epoch': 0.3}
{'loss': 0.2279, 'learning_rate': 3.944768552475059e-06, 'epoch': 0.3}
{'loss': 0.3479, 'learning_rate': 3.9433239591569385e-06, 'epoch': 0.31}
{'loss': 0.5752, 'learning_rate': 3.941860988490803e-06, 'epoch': 0.31}
{'loss': 0.3055, 'learning_rate': 3.940379654311464e-06, 'epoch': 0.31}
{'loss': 0.4535, 'learning_rate': 3.938879970627389e-06, 'epoch': 0.32}
{'loss': 0.2672, 'learning_rate': 3.937361951620575e-06, 'epoch': 0.32}
{'loss': 0.3706, 'learning_rate': 3.935825611646405e-06, 'epoch': 0.32}
{'loss': 0.184, 'learning_rate': 3.9342709652335196e-06, 'epoch': 0.33}
{'loss': 0.2241, 'learning_rate': 3.932698027083676e-06, 'epoch': 0.33}
{'loss': 0.4204, 'learn

{'loss': 0.2198, 'learning_rate': 3.7462227530175366e-06, 'epoch': 0.65}
{'loss': 0.3004, 'learning_rate': 3.74321608336278e-06, 'epoch': 0.65}
{'loss': 0.2341, 'learning_rate': 3.7401929287115407e-06, 'epoch': 0.66}
{'loss': 0.2731, 'learning_rate': 3.7371533176527566e-06, 'epoch': 0.66}
{'loss': 0.2912, 'learning_rate': 3.734097278930986e-06, 'epoch': 0.67}
{'loss': 0.2409, 'learning_rate': 3.731024841446138e-06, 'epoch': 0.67}
{'loss': 0.1748, 'learning_rate': 3.7279360342532006e-06, 'epoch': 0.67}
{'loss': 0.5325, 'learning_rate': 3.724830886561964e-06, 'epoch': 0.68}
{'loss': 0.2988, 'learning_rate': 3.721709427736744e-06, 'epoch': 0.68}
{'loss': 0.1883, 'learning_rate': 3.718571687296107e-06, 'epoch': 0.69}
{'loss': 0.3777, 'learning_rate': 3.7154176949125866e-06, 'epoch': 0.69}
{'loss': 0.2504, 'learning_rate': 3.7122474804124073e-06, 'epoch': 0.69}
{'loss': 0.1893, 'learning_rate': 3.7090610737752002e-06, 'epoch': 0.7}
{'loss': 0.1871, 'learning_rate': 3.7058585051337195e-06, '

{'loss': 0.1443, 'learning_rate': 3.4046135904602933e-06, 'epoch': 1.01}
{'loss': 0.1735, 'learning_rate': 3.4002286876912285e-06, 'epoch': 1.01}
{'loss': 0.191, 'learning_rate': 3.3958305434398194e-06, 'epoch': 1.02}
{'loss': 0.2829, 'learning_rate': 3.3914191992978063e-06, 'epoch': 1.02}
{'loss': 0.2467, 'learning_rate': 3.386994696981759e-06, 'epoch': 1.03}
{'loss': 0.1825, 'learning_rate': 3.382557078332677e-06, 'epoch': 1.03}
{'loss': 0.218, 'learning_rate': 3.3781063853155978e-06, 'epoch': 1.03}
{'loss': 0.2799, 'learning_rate': 3.3736426600191984e-06, 'epoch': 1.04}
{'loss': 0.1837, 'learning_rate': 3.3691659446553977e-06, 'epoch': 1.04}
{'loss': 0.277, 'learning_rate': 3.3646762815589566e-06, 'epoch': 1.05}
{'loss': 0.1635, 'learning_rate': 3.36017371318708e-06, 'epoch': 1.05}
{'loss': 0.2522, 'learning_rate': 3.3556582821190107e-06, 'epoch': 1.05}
{'loss': 0.1493, 'learning_rate': 3.351130031055633e-06, 'epoch': 1.06}
{'eval_loss': 0.24807609617710114, 'eval_content_rmse': 0.4

{'loss': 0.1201, 'learning_rate': 2.9513253613878983e-06, 'epoch': 1.37}
{'loss': 0.1914, 'learning_rate': 2.945910863629156e-06, 'epoch': 1.37}
{'loss': 0.2594, 'learning_rate': 2.9404874207158756e-06, 'epoch': 1.38}
{'loss': 0.1839, 'learning_rate': 2.935055083935696e-06, 'epoch': 1.38}
{'loss': 0.2286, 'learning_rate': 2.9296139046603625e-06, 'epoch': 1.39}
{'loss': 0.1462, 'learning_rate': 2.9241639343452406e-06, 'epoch': 1.39}
{'loss': 0.1459, 'learning_rate': 2.91870522452883e-06, 'epoch': 1.39}
{'loss': 0.1902, 'learning_rate': 2.913237826832278e-06, 'epoch': 1.4}
{'loss': 0.3618, 'learning_rate': 2.907761792958888e-06, 'epoch': 1.4}
{'loss': 0.1634, 'learning_rate': 2.9022771746936333e-06, 'epoch': 1.41}
{'loss': 0.376, 'learning_rate': 2.8967840239026683e-06, 'epoch': 1.41}
{'eval_loss': 0.22184258699417114, 'eval_content_rmse': 0.4092864394187927, 'eval_wording_rmse': 0.5255187749862671, 'eval_mcrmse': 0.4674026072025299, 'eval_runtime': 112.1676, 'eval_samples_per_second': 1

...

{'loss': 0.057, 'learning_rate': 1.309093203463029e-06, 'epoch': 2.45}
{'loss': 0.1227, 'learning_rate': 1.3033247813751897e-06, 'epoch': 2.45}
{'loss': 0.287, 'learning_rate': 1.2975629475058951e-06, 'epoch': 2.46}
{'loss': 0.1252, 'learning_rate': 1.2918077563428322e-06, 'epoch': 2.46}
{'loss': 0.1562, 'learning_rate': 1.2860592623108688e-06, 'epoch': 2.47}
{'eval_loss': 0.24141517281532288, 'eval_content_rmse': 0.3995905816555023, 'eval_wording_rmse': 0.5684691667556763, 'eval_mcrmse': 0.4840298891067505, 'eval_runtime': 112.1514, 'eval_samples_per_second': 18.341, 'eval_steps_per_second': 9.175, 'epoch': 2.47}
{'loss': 0.177, 'learning_rate': 1.280317519771541e-06, 'epoch': 2.47}
{'loss': 0.1554, 'learning_rate': 1.2745825830225386e-06, 'epoch': 2.47}
{'loss': 0.1294, 'learning_rate': 1.2688545062971906e-06, 'epoch': 2.48}
{'loss': 0.1616, 'learning_rate': 1.2631333437639536e-06, 'epoch': 2.48}
{'loss': 0.1065, 'learning_rate': 1.2574191495258992e-06, 'epoch': 2.49}
{'loss': 0.1039

{'loss': 0.2689, 'learning_rate': 8.126188539327434e-07, 'epoch': 2.81}
{'loss': 0.079, 'learning_rate': 8.076753395241966e-07, 'epoch': 2.82}
{'loss': 0.1171, 'learning_rate': 8.027431005209322e-07, 'epoch': 2.82}
{'eval_loss': 0.21399986743927002, 'eval_content_rmse': 0.39838746190071106, 'eval_wording_rmse': 0.518928587436676, 'eval_mcrmse': 0.45865803956985474, 'eval_runtime': 112.1448, 'eval_samples_per_second': 18.342, 'eval_steps_per_second': 9.176, 'epoch': 2.82}
{'loss': 0.0858, 'learning_rate': 7.978221835654409e-07, 'epoch': 2.82}
{'loss': 0.0567, 'learning_rate': 7.929126351931474e-07, 'epoch': 2.83}
{'loss': 0.0973, 'learning_rate': 7.880145018319657e-07, 'epoch': 2.83}
{'loss': 0.1124, 'learning_rate': 7.831278298018633e-07, 'epoch': 2.83}
{'loss': 0.1027, 'learning_rate': 7.782526653144204e-07, 'epoch': 2.84}
{'loss': 0.0991, 'learning_rate': 7.733890544723955e-07, 'epoch': 2.84}
{'loss': 0.0917, 'learning_rate': 7.685370432692888e-07, 'epoch': 2.85}
{'loss': 0.1233, 'le

{'eval_loss': 0.23560871183872223, 'eval_content_rmse': 0.41618937253952026, 'eval_wording_rmse': 0.5458974838256836, 'eval_mcrmse': 0.48104342818260193, 'eval_runtime': 112.1235, 'eval_samples_per_second': 18.346, 'eval_steps_per_second': 9.177, 'epoch': 3.17}
{'loss': 0.115, 'learning_rate': 4.0682613737563943e-07, 'epoch': 3.18}
{'loss': 0.0467, 'learning_rate': 4.0311564938117136e-07, 'epoch': 3.18}
{'loss': 0.128, 'learning_rate': 3.9942026257418914e-07, 'epoch': 3.18}
{'loss': 0.1363, 'learning_rate': 3.9574001190069705e-07, 'epoch': 3.19}
{'loss': 0.0899, 'learning_rate': 3.920749321635641e-07, 'epoch': 3.19}
{'loss': 0.0548, 'learning_rate': 3.8842505802219174e-07, 'epoch': 3.19}
{'loss': 0.0586, 'learning_rate': 3.8479042399218643e-07, 'epoch': 3.2}
{'loss': 0.0767, 'learning_rate': 3.811710644450372e-07, 'epoch': 3.2}
{'loss': 0.114, 'learning_rate': 3.775670136077842e-07, 'epoch': 3.21}
{'loss': 0.1741, 'learning_rate': 3.7397830556270104e-07, 'epoch': 3.21}
{'loss': 0.1317,

{'loss': 0.092, 'learning_rate': 1.373124610213201e-07, 'epoch': 3.53}
{'loss': 0.0759, 'learning_rate': 1.3508168074529568e-07, 'epoch': 3.53}
{'loss': 0.1275, 'learning_rate': 1.3286853636204674e-07, 'epoch': 3.54}
{'loss': 0.0724, 'learning_rate': 1.3067304880052211e-07, 'epoch': 3.54}
{'loss': 0.0853, 'learning_rate': 1.2849523882269386e-07, 'epoch': 3.54}
{'loss': 0.062, 'learning_rate': 1.2633512702336324e-07, 'epoch': 3.55}
{'loss': 0.0545, 'learning_rate': 1.2419273382996731e-07, 'epoch': 3.55}
{'loss': 0.0421, 'learning_rate': 1.2206807950238297e-07, 'epoch': 3.56}
{'loss': 0.1143, 'learning_rate': 1.1996118413273793e-07, 'epoch': 3.56}
{'loss': 0.074, 'learning_rate': 1.1787206764521785e-07, 'epoch': 3.56}
{'loss': 0.1107, 'learning_rate': 1.1580074979588217e-07, 'epoch': 3.57}
{'loss': 0.2115, 'learning_rate': 1.1374725017247233e-07, 'epoch': 3.57}
{'loss': 0.0537, 'learning_rate': 1.1171158819422943e-07, 'epoch': 3.57}
{'loss': 0.0621, 'learning_rate': 1.0969378311171152e-0

{'loss': 0.0552, 'learning_rate': 8.057713948828926e-09, 'epoch': 3.89}
{'loss': 0.0821, 'learning_rate': 7.515606045298772e-09, 'epoch': 3.89}
{'loss': 0.1344, 'learning_rate': 6.992340383151108e-09, 'epoch': 3.9}
{'loss': 0.1172, 'learning_rate': 6.48792191073011e-09, 'epoch': 3.9}
{'loss': 0.0511, 'learning_rate': 6.002355398147863e-09, 'epoch': 3.9}
{'loss': 0.0545, 'learning_rate': 5.535645437240833e-09, 'epoch': 3.91}
{'loss': 0.0856, 'learning_rate': 5.087796441524572e-09, 'epoch': 3.91}
{'loss': 0.1207, 'learning_rate': 4.658812646153531e-09, 'epoch': 3.92}
{'loss': 0.0935, 'learning_rate': 4.2486981078802e-09, 'epoch': 3.92}
{'loss': 0.0753, 'learning_rate': 3.857456705017359e-09, 'epoch': 3.92}
{'loss': 0.0957, 'learning_rate': 3.485092137400336e-09, 'epoch': 3.93}
{'loss': 0.1231, 'learning_rate': 3.1316079263534746e-09, 'epoch': 3.93}
{'loss': 0.1351, 'learning_rate': 2.7970074146557165e-09, 'epoch': 3.94}
{'eval_loss': 0.23408803343772888, 'eval_content_rmse': 0.4066893458

In [17]:
from pathlib import Path

seed = 42

fold = 3

output = f"output_fold{fold}_seed{seed}_2609"

!python train.py \
  --model_name_or_path "microsoft/deberta-v3-large" \
  --add_prompt_question True \
  --fold $fold \
  --data_dir "./" \
  --output_dir $output \
  --fp16 \
  --num_train_epochs 4 \
  --dataloader_num_workers 4 \
  --learning_rate 4e-6 \
  --weight_decay 1e-4 \
  --warmup_ratio 0 \
  --lr_scheduler_type "cosine" \
  --optim "adamw_torch" \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2 \
  --evaluation_strategy "steps" \
  --eval_steps 150 \
  --save_strategy "steps" \
  --save_steps 150 \
  --save_total_limit 1 \
  --report_to "wandb" \
  --metric_for_best_model "mcrmse" \
  --greater_is_better False \
  --logging_steps 10 \
  --log_level "error" \
  --disable_tqdm True \
  --ddp_find_unused_parameters False \
  --dropout 0 \
  --seed $seed


output_dir = Path.cwd() / output
# add json files
for json_file in output_dir.glob("checkpoint*/*token*.json"):
    json_file.rename(output_dir/json_file.name)

# model files
for model_file in output_dir.glob("checkpoint*/*model*"):
    model_file.rename(output_dir/model_file.name)

# remove optimizer states and other files
to_delete = str(list(output_dir.glob("checkpoint*"))[0])
!rm -r $to_delete

wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin
DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_probs_dropout_prob": 0.0,
  "cfg": {
    "add_prompt_question": true,
    "add_prompt_text": false,
    "data_dir": "./",
    "dropout": 0.0,
    "fold": 3,
    "max_position_embeddings": 1600,
    "max_seq_length": 1600,
    "model_name_or_path": "microsoft/deberta-v3-large",
    "num_proc": 4
  },
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 1600,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
 

{'loss': 0.2094, 'learning_rate': 3.950202929224252e-06, 'epoch': 0.29}
{'loss': 0.4465, 'learning_rate': 3.9488463922016285e-06, 'epoch': 0.29}
{'eval_loss': 0.3365912437438965, 'eval_content_rmse': 0.5444908142089844, 'eval_wording_rmse': 0.6137687563896179, 'eval_mcrmse': 0.5791298151016235, 'eval_runtime': 226.3707, 'eval_samples_per_second': 8.817, 'eval_steps_per_second': 4.409, 'epoch': 0.29}
{'loss': 0.3503, 'learning_rate': 3.947471864981005e-06, 'epoch': 0.29}
{'loss': 0.2375, 'learning_rate': 3.946079360250922e-06, 'epoch': 0.3}
{'loss': 0.3453, 'learning_rate': 3.9446688908658745e-06, 'epoch': 0.3}
{'loss': 0.2067, 'learning_rate': 3.943240469846193e-06, 'epoch': 0.31}
{'loss': 0.6987, 'learning_rate': 3.941794110377922e-06, 'epoch': 0.31}
{'loss': 0.2988, 'learning_rate': 3.9403298258127e-06, 'epoch': 0.31}
{'loss': 0.4119, 'learning_rate': 3.938847629667637e-06, 'epoch': 0.32}
{'loss': 0.2122, 'learning_rate': 3.937347535625187e-06, 'epoch': 0.32}
{'loss': 0.4549, 'learni

{'loss': 0.1867, 'learning_rate': 3.7521447665413904e-06, 'epoch': 0.64}
{'loss': 0.2351, 'learning_rate': 3.749206680113295e-06, 'epoch': 0.65}
{'loss': 0.1864, 'learning_rate': 3.746252446402045e-06, 'epoch': 0.65}
{'loss': 0.3838, 'learning_rate': 3.7432820926787746e-06, 'epoch': 0.65}
{'loss': 0.1519, 'learning_rate': 3.740295646363423e-06, 'epoch': 0.66}
{'loss': 0.2181, 'learning_rate': 3.737293135024485e-06, 'epoch': 0.66}
{'loss': 0.2199, 'learning_rate': 3.7342745863787545e-06, 'epoch': 0.67}
{'loss': 0.2549, 'learning_rate': 3.7312400282910684e-06, 'epoch': 0.67}
{'loss': 0.2283, 'learning_rate': 3.7281894887740515e-06, 'epoch': 0.67}
{'loss': 0.1753, 'learning_rate': 3.725122995987854e-06, 'epoch': 0.68}
{'loss': 0.2912, 'learning_rate': 3.7220405782398964e-06, 'epoch': 0.68}
{'loss': 0.245, 'learning_rate': 3.718942263984605e-06, 'epoch': 0.68}
{'loss': 0.1776, 'learning_rate': 3.715828081823148e-06, 'epoch': 0.69}
{'loss': 0.2262, 'learning_rate': 3.712698060503178e-06, 'e

{'loss': 0.2574, 'learning_rate': 3.4180753831300705e-06, 'epoch': 1.0}
{'loss': 0.3735, 'learning_rate': 3.413783817904206e-06, 'epoch': 1.0}
{'loss': 0.2139, 'learning_rate': 3.40947920175195e-06, 'epoch': 1.01}
{'loss': 0.359, 'learning_rate': 3.405161574410091e-06, 'epoch': 1.01}
{'loss': 0.1721, 'learning_rate': 3.400830975735526e-06, 'epoch': 1.01}
{'loss': 0.2926, 'learning_rate': 3.3964874457048933e-06, 'epoch': 1.02}
{'loss': 0.1549, 'learning_rate': 3.3921310244142038e-06, 'epoch': 1.02}
{'loss': 0.2888, 'learning_rate': 3.387761752078469e-06, 'epoch': 1.03}
{'loss': 0.0985, 'learning_rate': 3.383379669031333e-06, 'epoch': 1.03}
{'loss': 0.1747, 'learning_rate': 3.378984815724696e-06, 'epoch': 1.03}
{'loss': 0.1995, 'learning_rate': 3.374577232728344e-06, 'epoch': 1.04}
{'loss': 0.2438, 'learning_rate': 3.3701569607295737e-06, 'epoch': 1.04}
{'loss': 0.2271, 'learning_rate': 3.3657240405328165e-06, 'epoch': 1.04}
{'eval_loss': 0.20864902436733246, 'eval_content_rmse': 0.41301

{'loss': 0.2218, 'learning_rate': 2.974457799558738e-06, 'epoch': 1.35}
{'loss': 0.1521, 'learning_rate': 2.9691467875407218e-06, 'epoch': 1.36}
{'loss': 0.1476, 'learning_rate': 2.9638268291313068e-06, 'epoch': 1.36}
{'loss': 0.231, 'learning_rate': 2.9584979734401135e-06, 'epoch': 1.37}
{'loss': 0.1708, 'learning_rate': 2.9531602696588948e-06, 'epoch': 1.37}
{'loss': 0.2409, 'learning_rate': 2.9478137670610804e-06, 'epoch': 1.37}
{'loss': 0.1586, 'learning_rate': 2.942458515001326e-06, 'epoch': 1.38}
{'loss': 0.2089, 'learning_rate': 2.937094562915053e-06, 'epoch': 1.38}
{'loss': 0.5426, 'learning_rate': 2.931721960317996e-06, 'epoch': 1.38}
{'loss': 0.1551, 'learning_rate': 2.926340756805744e-06, 'epoch': 1.39}
{'loss': 0.2694, 'learning_rate': 2.9209510020532834e-06, 'epoch': 1.39}
{'eval_loss': 0.26857924461364746, 'eval_content_rmse': 0.49048900604248047, 'eval_wording_rmse': 0.5445906519889832, 'eval_mcrmse': 0.5175398588180542, 'eval_runtime': 227.2443, 'eval_samples_per_second

...

{'loss': 0.1166, 'learning_rate': 1.3525676057899405e-06, 'epoch': 2.42}
{'loss': 0.162, 'learning_rate': 1.3468212196127705e-06, 'epoch': 2.43}
{'loss': 0.0889, 'learning_rate': 1.3410808630619656e-06, 'epoch': 2.43}
{'loss': 0.0972, 'learning_rate': 1.3353465891279268e-06, 'epoch': 2.43}
{'loss': 0.1828, 'learning_rate': 1.3296184507449049e-06, 'epoch': 2.44}
{'eval_loss': 0.232829749584198, 'eval_content_rmse': 0.45194175839424133, 'eval_wording_rmse': 0.5112804770469666, 'eval_mcrmse': 0.48161113262176514, 'eval_runtime': 226.7511, 'eval_samples_per_second': 8.803, 'eval_steps_per_second': 4.401, 'epoch': 2.44}
{'loss': 0.1205, 'learning_rate': 1.3238965007905117e-06, 'epoch': 2.44}
{'loss': 0.1459, 'learning_rate': 1.3181807920852332e-06, 'epoch': 2.44}
{'loss': 0.1368, 'learning_rate': 1.3124713773919407e-06, 'epoch': 2.45}
{'loss': 0.1603, 'learning_rate': 1.3067683094154028e-06, 'epoch': 2.45}
{'loss': 0.0784, 'learning_rate': 1.3010716408018035e-06, 'epoch': 2.46}
{'loss': 0.1

{'loss': 0.144, 'learning_rate': 8.556141180649288e-07, 'epoch': 2.78}
{'loss': 0.126, 'learning_rate': 8.506358875522965e-07, 'epoch': 2.78}
{'loss': 0.1297, 'learning_rate': 8.456682670538964e-07, 'epoch': 2.79}
{'eval_loss': 0.25497105717658997, 'eval_content_rmse': 0.4522632360458374, 'eval_wording_rmse': 0.5526300668716431, 'eval_mcrmse': 0.5024466514587402, 'eval_runtime': 227.079, 'eval_samples_per_second': 8.79, 'eval_steps_per_second': 4.395, 'epoch': 2.79}
{'loss': 0.1155, 'learning_rate': 8.407113024268449e-07, 'epoch': 2.79}
{'loss': 0.1367, 'learning_rate': 8.357650394298932e-07, 'epoch': 2.79}
{'loss': 0.2818, 'learning_rate': 8.308295237230005e-07, 'epoch': 2.8}
{'loss': 0.1476, 'learning_rate': 8.259048008669189e-07, 'epoch': 2.8}
{'loss': 0.1595, 'learning_rate': 8.209909163227686e-07, 'epoch': 2.8}
{'loss': 0.1193, 'learning_rate': 8.160879154516165e-07, 'epoch': 2.81}
{'loss': 0.2052, 'learning_rate': 8.111958435140654e-07, 'epoch': 2.81}
{'loss': 0.1626, 'learning_r

{'eval_loss': 0.2188154011964798, 'eval_content_rmse': 0.42960238456726074, 'eval_wording_rmse': 0.5030629634857178, 'eval_mcrmse': 0.46633267402648926, 'eval_runtime': 226.7672, 'eval_samples_per_second': 8.802, 'eval_steps_per_second': 4.401, 'epoch': 3.13}
{'loss': 0.0801, 'learning_rate': 4.436705896925759e-07, 'epoch': 3.14}
{'loss': 0.0762, 'learning_rate': 4.3986131957537645e-07, 'epoch': 3.14}
{'loss': 0.0773, 'learning_rate': 4.3606645141611584e-07, 'epoch': 3.15}
{'loss': 0.0547, 'learning_rate': 4.322860202459935e-07, 'epoch': 3.15}
{'loss': 0.1356, 'learning_rate': 4.2852006096294155e-07, 'epoch': 3.15}
{'loss': 0.0799, 'learning_rate': 4.247686083312958e-07, 'epoch': 3.16}
{'loss': 0.1023, 'learning_rate': 4.2103169698148e-07, 'epoch': 3.16}
{'loss': 0.0672, 'learning_rate': 4.173093614096845e-07, 'epoch': 3.16}
{'loss': 0.0849, 'learning_rate': 4.1360163597754606e-07, 'epoch': 3.17}
{'loss': 0.1095, 'learning_rate': 4.09908554911834e-07, 'epoch': 3.17}
{'loss': 0.1037, 'l

{'loss': 0.073, 'learning_rate': 1.6227768479210658e-07, 'epoch': 3.49}
{'loss': 0.0877, 'learning_rate': 1.5988846699926328e-07, 'epoch': 3.49}
{'loss': 0.0448, 'learning_rate': 1.5751623565078732e-07, 'epoch': 3.49}
{'loss': 0.0774, 'learning_rate': 1.551610126452294e-07, 'epoch': 3.5}
{'loss': 0.1517, 'learning_rate': 1.5282281972413235e-07, 'epoch': 3.5}
{'loss': 0.1366, 'learning_rate': 1.5050167847183182e-07, 'epoch': 3.5}
{'loss': 0.0986, 'learning_rate': 1.4819761031525513e-07, 'epoch': 3.51}
{'loss': 0.0531, 'learning_rate': 1.4591063652372525e-07, 'epoch': 3.51}
{'loss': 0.0745, 'learning_rate': 1.436407782087621e-07, 'epoch': 3.52}
{'loss': 0.1472, 'learning_rate': 1.4138805632389138e-07, 'epoch': 3.52}
{'loss': 0.1209, 'learning_rate': 1.3915249166444688e-07, 'epoch': 3.52}
{'loss': 0.1067, 'learning_rate': 1.3693410486738133e-07, 'epoch': 3.53}
{'loss': 0.083, 'learning_rate': 1.347329164110753e-07, 'epoch': 3.53}
{'loss': 0.1448, 'learning_rate': 1.3254894661514793e-07, '

{'loss': 0.0925, 'learning_rate': 1.595390830688248e-08, 'epoch': 3.84}
{'loss': 0.0768, 'learning_rate': 1.5197073600689224e-08, 'epoch': 3.85}
{'loss': 0.0602, 'learning_rate': 1.4458561014587312e-08, 'epoch': 3.85}
{'loss': 0.0835, 'learning_rate': 1.3738377365936749e-08, 'epoch': 3.85}
{'loss': 0.0768, 'learning_rate': 1.3036529302899558e-08, 'epoch': 3.86}
{'loss': 0.0709, 'learning_rate': 1.2353023304378263e-08, 'epoch': 3.86}
{'loss': 0.0778, 'learning_rate': 1.1687865679955722e-08, 'epoch': 3.86}
{'loss': 0.1037, 'learning_rate': 1.1041062569837833e-08, 'epoch': 3.87}
{'loss': 0.1143, 'learning_rate': 1.0412619944795364e-08, 'epoch': 3.87}
{'loss': 0.0812, 'learning_rate': 9.802543606110436e-09, 'epoch': 3.88}
{'loss': 0.0823, 'learning_rate': 9.210839185522123e-09, 'epoch': 3.88}
{'loss': 0.0614, 'learning_rate': 8.637512145174275e-09, 'epoch': 3.88}
{'loss': 0.0433, 'learning_rate': 8.082567777565551e-09, 'epoch': 3.89}
{'eval_loss': 0.2117861956357956, 'eval_content_rmse': 0

In [18]:
from pathlib import Path
import json

seed = 42
scores = []
for fold in range(0,4):
    output = f"./output_fold{fold}_seed{seed}_2609"
    p = Path(output) / "config.json"

    with open(p) as fp:
        cfg = json.load(fp)


    print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
    scores.append(cfg['best_metric'])
print(sum(scores)/len(scores))

MCRMSE for fold 0, seed 42: 0.5513
MCRMSE for fold 1, seed 42: 0.4587
MCRMSE for fold 2, seed 42: 0.5038
MCRMSE for fold 3, seed 42: 0.4316
0.4863356798887253


#df.loc[0, 'prompt_text']

In [18]:
# from pathlib import Path
# import json

# seed = 42
# scores = []
# for fold in range(4):
#     output = f"./output_fold{fold}_seed{seed}_3008"
#     p = Path(output) / "config.json"

#     with open(p) as fp:
#         cfg = json.load(fp)


#     print(f"MCRMSE for fold {fold}, seed {seed}: {cfg['best_metric']:.4f}")
#     scores.append(cfg['best_metric'])
# print(sum(scores)/len(scores))